In [12]:
import cv2
import numpy as np
import imageio as io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
from sklearn.preprocessing import normalize,StandardScaler
import numpy as np
import argparse
import os
import random
from sklearn.model_selection import train_test_split
import skimage.io as io
import math
from skimage.filters import threshold_otsu
from skimage.filters import gaussian
from skimage.measure import label, regionprops
from skimage.color import rgb2gray,rgb2hsv
from skimage.util import invert
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.ndimage.morphology import binary_hit_or_miss
from scipy.signal import convolve2d
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.pyplot import bar
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from operator import eq
from sklearn import preprocessing
from skimage import color
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from skimage.io import imread, imshow

In [1]:
import cv2
import numpy as np
import imageio as io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,StandardScaler
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
import skimage.io as io
from skimage.feature import local_binary_pattern
from skimage import color
from operator import eq
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

In [2]:
def get_linehorizontalHaarfeatures(integral_image,window_height,window_width):
    linehorizontalHaarfeatures=[]
    window_width_third=int(window_width/3)
    i=0
    j=0
    while(i+window_height<integral_image.shape[0]):
        while(j+window_width<integral_image.shape[1]):
            white_Pixels_inHaar = integral_image[i+window_height][j+window_width_third]-integral_image[i+window_height][j]-integral_image[i][j+window_width_third]+integral_image[i][j]
            black_Pixels_inHaar = integral_image[i+window_height][j+window_width_third*2]-integral_image[i+window_height][j+window_width_third]-integral_image[i][j+window_width_third*2]+integral_image[i][j+window_width_third]
            white2_Pixels_inHaar = integral_image[i+window_height][j+window_width]-integral_image[i+window_height][j+window_width_third*2]-integral_image[i][j+window_width]+integral_image[i][j+window_width_third*2]
            delta=(white_Pixels_inHaar-black_Pixels_inHaar+white2_Pixels_inHaar)/(window_height*window_width_third)
            linehorizontalHaarfeatures.append(delta)
            j=j+1
        i=i+1  
    return linehorizontalHaarfeatures

In [3]:
def get_integralimage (img): 
    height,width=img.shape
    integral_image= np.zeros((height+1,width+1))#to add maargin 0 on top and on right
    for i in range(height):
        for j in range(width):
            integral_image[i][j]=img[i][j]+integral_image[i-1][j]+integral_image[i][j-1]-integral_image[i-1][j-1]#because we add the right top pixcel twice
    return integral_image                   

In [4]:
#Extarct Mouth using Haar Feature
def get_mouth(img,w=30,h=3):
    i=get_integralimage (img)
    integral_image=np.delete(i, img.shape[0],0)
    img=integral_image=np.delete(i,img.shape[1],1)    #to remove the margin
    im1=get_linehorizontalHaarfeatures(img,w,h)
    im1=np.array(im1)
    im1 = im1.reshape(5,9)
    return im1

In [5]:
# img  = io.imread('angry.png')
# neg_arr=np.zeros((img.shape[0],img.shape[1]), np.uint8)[0]
# # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# i=get_integralimage (img)
# integral_image=np.delete(i, img.shape[0],0)
# integral_image=np.delete(i,img.shape[1],1)    #to remove the margin
# # print(integral_image)
# im1=get_linehorizontalHaarfeatures(integral_image,40,3)
# im1=np.array(im1)
# print(np.array(im1).shape)
# im1 = im1.reshape(5,9)
# # neg_arr[:]=im1
# plt.imshow(im1,cmap="gray")
# plt.show()

In [6]:
# face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
# if face.empty():
#     raise IOError('Unable to load the face cascade classifier xml file')

In [7]:
# mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')
# if mouth_cascade.empty():
#     raise IOError('Unable to load the mouth cascade classifier xml file')

In [8]:
def extract_hog_features(img):
    img=imgmouth=get_mouth(img)
    if(np.array(imgmouth).any()==None):
        print("None")
    img = img.astype(int)
    target_img_size = (78, 78)
    img = cv2.resize(img.astype('uint8'), target_img_size)
    #64--.i.e 2 block
    win_size = (64, 64)
    cell_size = (8, 8)
    block_size_in_cells = (2, 2)

    block_size = (block_size_in_cells[1] * cell_size[1],
                  block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 15  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size,
                            block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()

    return h.flatten()

In [9]:
def preproccessing(img):
    gray=color.rgb2gray(img)
    return gray

In [10]:
# def extract_mouth(img):
#     pre_img=preproccessing(img)
#     original_image=pre_img
#     pre_img = np.array(pre_img, dtype='uint8')
#     mouth_rects = mouth_cascade.detectMultiScale(pre_img,1.2, 10)
#     for (x,y,w,h) in mouth_rects:
#         #print(x,y,w,h)
#         y = int(y - 0.15*h)
#         #cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255),1)
#         cropped_contour= original_image[y:y+h, x:x+w]
#         image_name= "output_shape_number.jpg"
#         cv2.imwrite(image_name, cropped_contour)
        
#         readimage= cv2.imread(image_name) 
#         return readimage

In [11]:
# path_to_dataset =r'D:\Download\jaffedbase'
path_to_dataset =r'E:\emotion\archive\ck\CK+48'
# path_to_dataset =r'E:\datasetemotion\val\val'
img_filenames = os.listdir(path_to_dataset)

In [12]:
img_filenames = os.listdir(path_to_dataset)
print(img_filenames)

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']


In [13]:
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)

In [14]:
def load_dataset(feature_set='hog'):
    
    features = []
    labels = []

    for i, fn in enumerate(img_filenames):
            label = fn.split('.')[0]
            path = os.path.join(path_to_dataset, fn)
            subpath=os.listdir(path)
            for k, d in enumerate(subpath):             
                imgpath = os.path.join(path, d)
                img = io.imread(imgpath)
                img = cv2.resize(img,(48,48), interpolation = cv2.INTER_AREA)
                img=color.rgb2gray(img)
                if(np.array(extract_features(img)).any()!=None):
                    labels.append(label)
                    features.append(extract_features(img))
                if k > 0 and k % 100 == 0:
                    print("[INFO] processed {}/{}".format(k, len(subpath)))
    return features, labels            

In [15]:
def extract_features(img):
    return extract_hog_features(img) 

In [16]:
def split_data(features,labels,precentage):
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    return train_features, test_features, train_labels, test_labels

In [17]:
def KNN_classifier(kcount,train_features, train_labels):
    classifier = KNeighborsClassifier(n_neighbors=kcount)
    classifier.fit(list(train_features),list(train_labels) )
    return classifier

In [18]:
def normalization(train_features, test_features):
    scaler = StandardScaler()
    scaler.fit(train_features)
    train_features = scaler.transform(train_features)
    test_features = scaler.transform(test_features)
    return train_features, test_features

In [19]:
def test_classifier(test_features, test_labels,classifier):
        accuracy = classifier.score(test_features, test_labels)        
        print('accuracy:', accuracy*100, '%')

In [20]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset(feature_set='hog')

print('Finished loading dataset.')

Loading dataset. This will take time ...
[INFO] processed 100/135
[INFO] processed 100/177
[INFO] processed 100/207
[INFO] processed 200/207
[INFO] processed 100/249
[INFO] processed 200/249
Finished loading dataset.


In [21]:
features1, labels1 =features, labels 

In [22]:
print(np.array(features1).shape)
print(np.array(labels1).shape)

(981, 11760)
(981,)


In [23]:
train_features, test_features, train_labels, test_labels=split_data(np.array(features1),np.array(labels1),0.2)

In [24]:
from sklearn.metrics.pairwise import rbf_kernel
le = preprocessing.LabelEncoder()
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)
sigma=0.0008
c=64
reg=100
err=1e-4
#Get Kernel Function
def kernel(X, Y=None,g=None):
    k=rbf_kernel(X, Y, gamma=g)
    return k
#Get Output Weight "Fitting Classifier"
def KELM_Beta(sigma,c,x,y):
    le.fit(y)
    y = le.transform(y)
    emotion = len(np.unique(y))
    One_Hot_Encoding=np.zeros((emotion,emotion), np.uint8)
    for em in range (emotion):
        One_Hot_Encoding[em][em]=1
    T = One_Hot_Encoding[y, :]    
    N,d=x.shape
    Omega=kernel(x,None,sigma)
    I=np.eye(N)
    Beta = np.linalg.inv((I /c) + Omega).dot(T)    
    return Beta,One_Hot_Encoding
#Get Predicted Labels "Predication"
def KELM_Output(sigma,x,x_test,Beta,One_Hot_Encoding):
    hx=kernel(x_test,x,sigma)
    fx= hx.dot(Beta)
    p=np.argmax(fx,axis=1)
    return p
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)    
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(y_pred)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc
Beta,One_Hot_Encoding=KELM_Beta(sigma,c,np.array(train_features),np.array(train_labels))
p1=KELM_Output(sigma,np.array(train_features),np.array(test_features),Beta,One_Hot_Encoding)
print("test",acc(p1,test_labels))

p2=KELM_Output(sigma,np.array(train_features),np.array(train_features),Beta,One_Hot_Encoding)
print("train",acc(p2,train_labels))

test 88.3248730964467
train 99.87244897959184


In [25]:
#KNN

In [26]:
# print(train_features)
print(np.array(train_features).shape)
print(np.array(train_labels).shape)

(784, 11760)
(784,)


In [27]:
classifier=KNN_classifier(3,train_features, train_labels)


In [28]:
y_pred = classifier.predict(test_features)

In [32]:
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(yt)))
    size=len(rt)
    acc=(eqw/size)*100
    print(acc)
    print(yt)
    print(rt)
    
    return acc


In [33]:
accuracy = classifier.score(test_features, test_labels)
print('Test accuracy:', accuracy*100, '%')
accuracy = classifier.score(train_features, train_labels)
print('Train accuracy:', accuracy*100, '%')

Test accuracy: 72.08121827411168 %
Train accuracy: 95.53571428571429 %


In [34]:
print(accuracy_score(test_labels, y_pred))
print(classification_report(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred))

0.7208121827411168
              precision    recall  f1-score   support

       anger       0.65      0.85      0.74        20
    contempt       0.44      0.67      0.53         6
     disgust       0.71      0.66      0.68        41
        fear       0.80      0.80      0.80        15
       happy       0.70      0.85      0.77        39
     sadness       0.67      0.55      0.60        22
    surprise       0.84      0.69      0.76        54

    accuracy                           0.72       197
   macro avg       0.69      0.72      0.70       197
weighted avg       0.73      0.72      0.72       197

[[17  0  1  0  1  1  0]
 [ 0  4  0  0  2  0  0]
 [ 6  0 27  0  7  0  1]
 [ 0  0  0 12  0  1  2]
 [ 1  0  2  0 33  1  2]
 [ 0  4  4  0  0 12  2]
 [ 2  1  4  3  4  3 37]]


In [102]:
import joblib
# joblib.dump(classifier, 'DellKNN2.pkl')


In [103]:
# img  = io.imread('KA.AN2.40.tiff')
img  = io.imread('MK.HA1.116.tiff')
# img  = io.imread('UY.DI2.150.tiff')



f=extract_features(img)
print(f)

[0.18610714 0.21370839 0.15028238 ... 0.21837309 0.09984857 0.10134899]


In [104]:
y_pred = classifier.predict([f] )
print(y_pred)

['Happy']


In [105]:
#SVM

In [35]:
from sklearn.svm import SVC
# k in ['rbf','sigmoid']\
# gamma in ['scale','auto']

k='sigmoid'
gamma = 'auto'
c=400
clf = SVC(C=c,kernel=k,gamma=gamma)
clf.fit(train_features, train_labels)
print ('train score', clf.score(train_features, train_labels)*100 )
y=clf.predict(test_features)
print ('test score', clf.score(test_features,test_labels)*100 )

train score 95.28061224489795
test score 80.20304568527918


In [52]:
print(accuracy_score(test_labels, y))
print(classification_report(test_labels, y))
print(confusion_matrix(test_labels, y))

0.9583333333333334
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00         0
    contempt       1.00      0.33      0.50         3
     disgust       1.00      1.00      1.00         7
        fear       1.00      1.00      1.00         1
       happy       0.97      1.00      0.99        36
    surprise       1.00      1.00      1.00         1

    accuracy                           0.96        48
   macro avg       0.83      0.72      0.75        48
weighted avg       0.98      0.96      0.96        48

[[ 0  0  0  0  0  0]
 [ 1  1  0  0  1  0]
 [ 0  0  7  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0 36  0]
 [ 0  0  0  0  0  1]]


E:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
# img  = io.imread('KA.AN2.40.tiff')
img  = io.imread('UY.DI2.150.tiff')
# img  = io.imread('KL.SU1.164.tiff')
# img  = io.imread('S010_004_00000017.png')
# img  = io.imread('MK.HA1.116.tiff')
f=extract_features(img)
print(f)

[0.05678637 0.03077303 0.04064067 ... 0.09820569 0.14047983 0.19829692]


In [54]:
y_predsvm=clf.predict([f])
print(y_predsvm)

['happy']


In [83]:
import joblib
# joblib.dump(clf, 'DellSVM.pkl')

['DellSVM.pkl']